### Loading and Viewing the Dataset

In [1]:
# import libraries
import pandas as pd
import numpy as np

# load the datasets
retail_data = pd.read_csv('retail_data_W23 - retail_data_W23.csv')
retail_orders = pd.read_csv('retail_orders_W23 - retail_orders_W23.csv')
store_data = pd.read_csv('store - store.csv')

In [4]:
# store dataframes in a dictionary
dataframes = {'retail_data': retail_data, 'retail_orders': retail_orders, 'store_data': store_data}

# view first 5 rows and info
for name, df in dataframes.items():
    print(f"\n{name} DataFrame - First 5 Rows:")
    print(df.head()) 
    
    print(f"\n{name} DataFrame - Info:")
    print(df.info())


retail_data DataFrame - First 5 Rows:
   Store  DayOfWeek        Date  Customers  Open  Promo StateHoliday  \
0      3          5  2015-07-31        821     1      1            0   
1      4          5  2015-07-31       1498     1      1            0   
2      5          5  2015-07-31        559     1      1            0   
3      6          5  2015-07-31        589     1      1            0   
4     10          5  2015-07-31        681     1      1            0   

   SchoolHoliday   Id  
0              1  409  
1              1  427  
2              1  445  
3              1  463  
4              1  535  

retail_data DataFrame - Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651013 entries, 0 to 651012
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Store          651013 non-null  int64 
 1   DayOfWeek      651013 non-null  int64 
 2   Date           651013 non-null  object
 3   Customers      651

### Merging data

In [5]:
merged_data = pd.merge(retail_data, store_data, on='Store', how='left')
merged_data = pd.merge(merged_data, retail_orders, on='Id', how='left')

### Exploratory Data Analysis

In [8]:
print(merged_data.describe())

               Store      DayOfWeek      Customers           Open  \
count  651013.000000  651013.000000  651013.000000  651013.000000   
mean      558.645629       3.999336     632.851832       0.829619   
std       321.905872       1.998260     464.857658       0.375967   
min         1.000000       1.000000       0.000000       0.000000   
25%       281.000000       2.000000     404.000000       1.000000   
50%       558.000000       4.000000     609.000000       1.000000   
75%       838.000000       6.000000     837.000000       1.000000   
max      1115.000000       7.000000    5458.000000       1.000000   

               Promo  SchoolHoliday            Id  CompetitionDistance  \
count  651013.000000  651013.000000  6.510130e+05        649312.000000   
mean        0.381558       0.178927  9.153619e+06          5436.342390   
std         0.485769       0.383292  5.285944e+06          7713.881629   
min         0.000000       0.000000  4.090000e+02            20.000000   
25%     

In [10]:
# check for missing values
print(merged_data.isna().sum())

Store                             0
DayOfWeek                         0
Date                              0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
Id                                0
StoreType                         0
Assortment                        0
CompetitionDistance            1701
CompetitionOpenSinceMonth    207162
CompetitionOpenSinceYear     207162
Promo2                            0
Promo2SinceWeek              325446
Promo2SinceYear              325446
PromoInterval                325446
Orders                            0
dtype: int64


In [11]:
# Filter rows where CompetitionDistance is missing
missing_competition_distance = merged_data.loc[merged_data['CompetitionDistance'].isna()]

# Display those rows with all columns
print(missing_competition_distance)

        Store  DayOfWeek        Date  Customers  Open  Promo StateHoliday  \
406       622          5  2015-07-31        540     1      1            0   
566       879          5  2015-07-31        466     1      1            0   
915       291          4  2015-07-30        883     1      1            0   
1126      622          4  2015-07-30        406     1      1            0   
1297      879          4  2015-07-30        426     1      1            0   
...       ...        ...         ...        ...   ...    ...          ...   
649270    622          4  2013-01-03        419     1      0            0   
649435    879          4  2013-01-03        332     1      0            0   
649968    622          3  2013-01-02        451     1      0            0   
650133    879          3  2013-01-02        326     1      0            0   
650482    291          2  2013-01-01          0     0      0            a   

        SchoolHoliday        Id StoreType Assortment  CompetitionDistance  